# 5年营业收入增长率 sales_growth
过去5个财年的 每股营业收入增长 除以 年均每股营业收入

注意：对于基本面数据，
每年11-第二年4月使用前一年0930的TTM数据，既三季报+前一年四季度
5-8月使用1季度0331的TTM数据，
9-10月使用中报0630的TTM数据，
11-12月使用当年三季报0930的TTM数据

In [1]:
import datetime

import pandas as pd
# 获取tushare每股营业收入数据
import tushare as ts

pro = ts.pro_api('e9628663b45f87ea92e39aaa7127063830ebd5090bc1e8943138b84f')



In [6]:
datetime.datetime.today().strftime('%Y%m%d')

'20230403'

In [3]:
df = pro.fina_indicator_vip(period='20221231')
df

,ts_code,ann_date,end_date,eps,dt_eps,total_revenue_ps,revenue_ps,capital_rese_ps,surplus_rese_ps,undist_profit_ps,...,roe_yoy,bps_yoy,assets_yoy,eqt_yoy,tr_yoy,or_yoy,q_sales_yoy,q_op_qoq,equity_yoy,update_flag
0,A22231.SZ,20230118,20221231,4.1400,4.0500,27.3891,27.3891,1.4895,0.5000,6.4809,...,62.8621,74.3120,62.2231,74.3135,58.6078,58.6078,NaN,NaN,74.3135,1
1,A22460.SZ,20230118,20221231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.1659,NaN,45.6121,25.5637,91.0482,91.0482,NaN,NaN,25.5637,1
2,A22278.SZ,20230130,20221231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-11.0430,NaN,30.7480,41.9701,21.4938,21.4938,NaN,NaN,41.9701,0
3,A22137.BJ,20230201,20221231,0.4500,0.4500,13.0174,13.0174,1.6935,0.1906,0.8117,...,4.2923,4.7087,-1.5606,4.7099,15.8528,15.8528,40.7736,477.0235,4.7099,1
4,A21410.SZ,20230310,20221231,0.2700,0.2700,2.6092,2.6092,0.2330,0.0900,0.4380,...,43.0436,17.3300,4.2451,17.7526,36.9964,36.9964,3.2901,41.0361,17.7526,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1736,300316.SZ,20230403,20221231,2.2600,2.2600,8.1288,8.1288,2.0191,0.5866,4.6671,...,8.3653,54.9378,71.0915,57.6168,78.4544,78.4544,61.2132,9.5864,57.6168,1
1737,301175.SZ,20230403,20221231,0.1700,0.1700,1.0849,1.0849,0.7649,0.0234,0.3938,...,-37.1862,46.3876,42.3858,94.9694,5.3443,5.3443,0.0715,-36.4806,94.9694,1
1738,605167.SH,20230403,20221231,0.3100,0.3100,3.8326,3.8326,0.9449,0.1039,1.2538,...,15.1807,8.7442,24.4590,8.7436,-13.2092,-13.2092,22.7419,82.4347,8.7436,1
1739,688410.SH,20230403,20221231,0.5458,0.5458,2.6395,2.6395,9.2554,0.0919,0.1177,...,-22.3375,193.7461,246.0290,291.6896,34.8549,34.8549,30.7263,28.4906,291.6896,1


In [10]:
# 获取tusahre财务指标数据，可指定数据品种、证券范围以及数据获取时间
def get_fin_indicators(period, indicators, security=[], date=datetime.datetime.today().strftime('%Y%m%d'),
                       market_sign='A'):
    '''
    :param security: security list
    :param date: yyyymmdd
    :param period: yyyymmdd
    :param indicators: list
    :return: dataframe
    '''

    fields = ['ts_code', 'ann_date', 'end_date', 'update_flag']
    fields.extend(indicators)
    revenue_ps_df = pro.fina_indicator_vip(period=period, fields=fields)
    revenue_ps_df.dropna(inplace=True)
    # 查看是否指定了证券区间
    if len(security) > 0:
        revenue_ps_df = revenue_ps_df.loc[revenue_ps_df['ts_code'].isin(security)]
    if market_sign == 'A':
        revenue_ps_df = revenue_ps_df[revenue_ps_df['ts_code'].str.startswith(('0', '3', '6'))]
    revenue_ps_df = revenue_ps_df[revenue_ps_df['ann_date'] <= date]
    revenue_ps_df = revenue_ps_df.reset_index()
    revenue_ps_df.drop('index', axis=1)
    return revenue_ps_df

# get_fin_indicators(date='20230316', period='20221231', indicators=['eps', 'revenue_ps'])

In [11]:
# 生成revenue-ps的TTM数据
period = '20221231'
indicators = ['eps', 'revenue_ps']
get_fin_indicators(period, indicators)

,index,ts_code,ann_date,end_date,eps,revenue_ps,update_flag
0,0,600873.SH,20220715,20220630,0.8300,4.5890,0
1,1,600873.SH,20220715,20220630,0.8300,4.5890,1
2,2,300463.SZ,20220716,20220630,0.6728,2.9092,0
3,4,002107.SZ,20220716,20220630,0.1000,0.8725,0
4,6,300463.SZ,20220716,20220630,0.6728,2.9092,1
...,...,...,...,...,...,...,...
5979,7863,688539.SH,20230329,20220630,0.3900,1.3218,1
5980,7864,688146.SH,20230331,20220630,0.4600,2.1227,1
5981,7866,688478.SH,20230331,20220630,0.0262,0.6269,1
5982,7870,301360.SZ,20230331,20220630,0.3100,2.6190,1


In [45]:
# 生成revenue-ps的TTM数据
indicators = ['eps', 'revenue_ps']
period = '20220930'
period_list = ['20220930','20211231','20210930']
df = get_fin_indicators(period, indicators)
selected_sec_ls = df.ts_code.tolist()
for period in period_list[1:]:
    new_df = get_fin_indicators(period,indicators)
    selected_sec_ls = list(set(selected_sec_ls)&set(new_df.ts_code.tolist()))
    df = pd.concat([df,new_df],ignore_index=True)
df = df.loc[df['ts_code'].isin(selected_sec_ls)]
df = df.loc[df['ts_code'].isin(selected_sec_ls)]

In [43]:
df = df.reset_index()

In [46]:
df

,index,ts_code,ann_date,end_date,eps,revenue_ps,update_flag
0,0,603185.SH,20221010,20220930,7.3700,44.7846,1
1,1,600557.SH,20221010,20220930,0.4900,5.3464,1
2,2,603300.SH,20221011,20220930,0.3500,1.8405,1
3,3,603083.SH,20221011,20220930,0.2900,11.1014,1
4,4,002772.SZ,20221012,20220930,0.3055,3.5462,1
...,...,...,...,...,...,...,...
23450,10017,688525.SH,20221213,20210930,0.3200,5.2833,1
23451,10041,001223.SZ,20221129,20210930,3.7300,8.4468,1
23452,10052,301206.SZ,20220114,20210930,4.3600,12.9036,1
23453,10065,600927.SH,20211109,20210930,0.6700,20.8916,1


In [32]:
origin_sec_ls = df.ts_code.tolist()
selected_sec_ls = []
for sec in origin_sec_ls:
    if origin_sec_ls.count(sec)==3:
        selected_sec_ls.append(sec[:6])
selected_sec_ls = list(set(selected_sec_ls))



In [35]:
selected_sec_ls

['600747',
 '688325',
 '301311',
 '301248',
 '301152',
 '688293',
 '688052',
 '600734',
 '301439',
 '300038',
 '688091',
 '600155',
 '600115',
 '301265',
 '301162',
 '002470',
 '001300',
 '301377',
 '301105',
 '688337',
 '600928',
 '688141',
 '688132',
 '301226',
 '601838',
 '301153',
 '001338',
 '688279',
 '301396',
 '601229',
 '301263',
 '001314',
 '603030',
 '301028',
 '001289',
 '603272',
 '688273',
 '301126',
 '600925',
 '300500',
 '001323',
 '600614',
 '688193',
 '301238',
 '600901',
 '688048',
 '001238',
 '301365',
 '601288',
 '688047',
 '002298',
 '000063',
 '300752',
 '000832',
 '001258',
 '601089',
 '000783',
 '688371',
 '688209',
 '301183',
 '688150',
 '601688',
 '688376',
 '603052',
 '301339',
 '603363',
 '688385',
 '300372',
 '688432',
 '001298',
 '002961',
 '688192',
 '600109',
 '603211',
 '000625',
 '600069',
 '301258',
 '603280',
 '000687',
 '002619',
 '688503',
 '600112',
 '301313',
 '300517',
 '688275',
 '600677',
 '603209',
 '601577',
 '603051',
 '600938',
 '301257',

In [ ]:
target_df = pd.DataFrame(columns = df.columns.tolist())

In [163]:

from datetime import datetime


# 生成报告期序列
def get_report_periods_list(end_date=datetime.today(), n_years=5):
    today = end_date
    dates = []
    for i in range(0, n_years + 2):
        year = today.year - i
        dates.append(datetime(year, 3, 31).strftime('%Y%m%d'))
        dates.append(datetime(year, 6, 30).strftime('%Y%m%d'))
        dates.append(datetime(year, 9, 30).strftime('%Y%m%d'))
        dates.append(datetime(year, 12, 31).strftime('%Y%m%d'))
    dates.sort()
    dates = [x for x in dates if x < today.strftime('%Y%m%d')][n_years * 4 * (-1):]
    return dates


period_list = get_report_periods_list(end_date=datetime.today(), n_years=5)

In [164]:
len(period_list)

20

In [119]:
# 根据报告期列表获取单季数据,这里指单季营业收入同比增速
date = datetime.today().strftime('%Y%M%D')
df = get_fin_indicators(date=date, period=period_list[0], indicators=['eps', 'or_yoy'])
for period in period_list[1:]:
    df = pd.concat([df, get_fin_indicators(date=date, period=period, indicators=['eps', 'or_yoy'])], ignore_index=True)
df

,index,ts_code,ann_date,end_date,eps,or_yoy,update_flag
0,1,000563.SZ,20180428,20180331,0.0396,11.8520,0
1,2,000022.SZ,20180427,20180331,0.2270,4.2839,1
2,5,600087.SH,20180424,20180331,0.0179,-12.4245,1
3,6,000043.SZ,20180428,20180331,0.1013,51.8616,1
4,7,300571.SZ,20180426,20180331,0.6000,12.3746,0
...,...,...,...,...,...,...,...
112990,951,300602.SZ,20230328,20221231,0.1900,34.8757,1
112991,952,000728.SZ,20230328,20221231,0.4000,-12.5816,1
112992,953,301387.SZ,20230328,20221231,1.9900,0.0342,1
112993,954,688669.SH,20230328,20221231,0.4800,55.6982,1


In [129]:
negative_eps_sec = list(set(df.loc[df['eps'] < 0, 'ts_code'].tolist()))

In [120]:
sec_list = list(set(df.ts_code.tolist()))
indicator_list = []
for sec in sec_list:
    single_list = df.loc[df['ts_code'] == sec, 'or_yoy'].tolist()
    indicator_list.append(single_list)

In [121]:
indicator_dict = dict(zip(sec_list, indicator_list))
# 删除长度不足 15 的 list
indicator_dict = {k: v for k, v in indicator_dict.items() if len(v) >= 15}
# 对每个 list 去掉一个最大值和一个最小值
indicator_dict = {k: sorted(v)[2:-2] for k, v in indicator_dict.items()}

In [122]:
new_dict = {k: sum(v) / len(v) for k, v in indicator_dict.items()}
mean_df = pd.DataFrame(list(new_dict.items()), columns=['ts_code', 'Mean'])

In [130]:
mean_df = mean_df.loc[~mean_df['ts_code'].isin(negative_eps_sec)]

In [131]:
mean_df

,ts_code,Mean
0,688698.SH,35.823969
1,002455.SZ,23.041495
2,002331.SZ,-6.245710
3,605179.SH,14.288475
4,002520.SZ,5.214139
...,...,...
4407,002712.SZ,14.570586
4408,300517.SZ,42.465900
4409,300297.SZ,-38.807175
4410,300614.SZ,6.838593


In [146]:
def years_revenue_growth(date=date, n=5, indicators=['eps', 'or_yoy']):
    period_list = get_report_periods_list(end_date=date, n_years=5)
    # 根据报告期列表获取单季数据,这里指单季营业收入同比增速
    df = get_fin_indicators(date=date, period=period_list[0], indicators=indicators)
    for period in period_list[1:]:
        df = pd.concat([df, get_fin_indicators(date=date, period=period, indicators=indicators)],
                       ignore_index=True)
    negative_eps_sec = set(df.loc[df['eps'] < 0, 'ts_code'].tolist())  # 出现过亏损的标的
    sec_list = list(set(df.ts_code.tolist()) - negative_eps_sec)  # 出去出现过亏损的标的
    indicator_list = []
    for sec in sec_list:
        single_list = df.loc[df['ts_code'] == sec, 'or_yoy'].tolist()
        indicator_list.append(single_list)
    indicator_dict = dict(zip(sec_list, indicator_list))
    # 删除长度不足 15 的 list
    indicator_dict = {k: v for k, v in indicator_dict.items() if len(v) >= 15}
    # 对每个 list 去掉n个最大值和n个最小值
    indicator_dict = {k: sorted(v)[2:-2] for k, v in indicator_dict.items()}
    new_dict = {k: sum(v) / len(v) for k, v in indicator_dict.items()}
    mean_df = pd.DataFrame(list(new_dict.items()), columns=['ts_code', 'Mean'])
    mean_df = mean_df.loc[~mean_df['ts_code'].isin(negative_eps_sec)]
    df_sorted = mean_df.sort_values(by='Mean', ascending=False)
    df_sorted = df_sorted.reset_index(drop=True)
    return df_sorted


date = datetime.today().strftime('%Y%M%D')
years_average_revenue_growth_df = years_revenue_growth(date=date, n=5, indicators=['eps', 'or_yoy'])

In [156]:
#筛选收入增速均值处于最大10%的证券
q = years_average_revenue_growth_df['Mean'].quantile(q=0.9)
selected_revenue_growth_df = years_average_revenue_growth_df[years_average_revenue_growth_df['Mean'] >= q]
selected_revenue_growth_securities = selected_revenue_growth_df.ts_code.tolist()
selected_revenue_growth_securities

In [167]:
# 获取这些证券的市值
type(selected_revenue_growth_securities)

list

In [179]:
date = datetime.today().strftime('%Y%M%D')
df = pro.daily_basic(ts_code='', trade_date='20230328',
                     fields=['ts_code', 'trade_date', 'turnover_rate', 'volume_ratio', 'pe', 'pb', 'circ_mv'])

In [181]:
df = df[df['ts_code'].isin(selected_revenue_growth_securities)]

In [187]:
df_sorted = df.sort_values(by='circ_mv', ascending=True)
df_sorted = df_sorted.reset_index(drop=True)

In [188]:
df_sorted

,ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,circ_mv
0,605180.SH,20230328,2.3328,None,11.5294,1.8968,5.980000e+04
1,300984.SZ,20230328,1.7632,None,28.3216,2.7866,6.892484e+04
2,003036.SZ,20230328,2.3378,None,22.1364,2.2830,7.564614e+04
3,300930.SZ,20230328,2.2973,None,27.8799,3.4433,7.911900e+04
4,688345.SH,20230328,4.0212,None,35.1222,3.6019,1.065842e+05
...,...,...,...,...,...,...,...
220,688111.SH,20230328,1.7473,None,165.5186,21.1972,1.849719e+07
221,002475.SZ,20230328,1.1510,None,29.9901,5.0801,2.116959e+07
222,300059.SZ,20230328,1.9643,None,30.9489,4.0414,2.209972e+07
223,601012.SH,20230328,0.5629,None,33.2519,5.2336,3.019845e+07
